In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:
data_fillna.drop('Description', axis=1, inplace=True)
data_fillna.drop('Start_Time', axis=1, inplace=True)
data_fillna.drop('End_Time', axis=1, inplace=True)
data_fillna.drop('Unnamed: 0', axis=1, inplace=True)
data_fillna.drop('Street', axis=1, inplace=True)
data_fillna.drop('Airport_Code', axis=1, inplace=True)
data_fillna.drop('Weather_Timestamp', axis=1, inplace=True)

In [ ]:
train_df = pd.read_csv('US_Accidents_Dec21_updated.csv')
train_df.head()

In [ ]:
data_fillna = train_df.fillna(method='ffill', axis=0)

In [ ]:
data_fillna.to_csv("Dropped.csv")

In [ ]:
train_df = pd.read_csv('Dropped.csv')
train_df.head()

,Unnamed: 0,ID,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,City,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,0,A-1,3,40.108910,-83.092860,40.112060,-83.031870,3.230,R,Dublin,...,False,False,False,False,False,False,Night,Night,Night,Night
1,1,A-2,2,39.865420,-84.062800,39.865010,-84.048730,0.747,R,Dayton,...,False,False,False,False,False,False,Night,Night,Night,Night
2,2,A-3,2,39.102660,-84.524680,39.102090,-84.523960,0.055,R,Cincinnati,...,False,False,False,False,False,False,Night,Night,Night,Day
3,3,A-4,2,41.062130,-81.537840,41.062170,-81.535470,0.123,R,Akron,...,False,False,False,False,False,False,Night,Night,Day,Day
4,4,A-5,3,39.172393,-84.492792,39.170476,-84.501798,0.500,R,Cincinnati,...,False,False,False,False,False,False,Day,Day,Day,Day


In [ ]:

def split_zip(col):
    zip_list = []
    for zip in col:
        zip_first = zip.split("-")[0]
        zip_list.append(zip_first)
    data_fillna["Zipcode"] = zip_list

split_zip(data_fillna["Zipcode"])

In [ ]:
# Replace nan with most occuring
train_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
train_df.head()

,ID,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,City,County,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,40.108910,-83.092860,40.112060,-83.031870,3.230,R,Dublin,Franklin,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,39.865420,-84.062800,39.865010,-84.048730,0.747,R,Dayton,Montgomery,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,39.102660,-84.524680,39.102090,-84.523960,0.055,R,Cincinnati,Hamilton,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,41.062130,-81.537840,41.062170,-81.535470,0.123,R,Akron,Summit,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,39.172393,-84.492792,39.170476,-84.501798,0.500,R,Cincinnati,Hamilton,...,False,False,False,False,False,False,Day,Day,Day,Day


In [ ]:
## Create our Text Vectorizer to index our vocabulary based on the train samples 
import tensorflow as tf
from keras.layers import TextVectorization

def vectorize_data(data):
    vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=3)
    text_ds = tf.data.Dataset.from_tensor_slices(data).batch(128) ## Read batches of 128 samples
    vectorizer.adapt(text_ds)
    ## Create a map to get the unique list of the vocabulary
    voc = vectorizer.get_vocabulary()
    word_index = dict(zip(voc, range(len(voc))))
    
    x_train_data = vectorizer(np.array([[s] for s in data])).numpy()
    return x_train_data
    
cities = vectorize_data(train_df["City"])
train_df["City"] = cities[:, 0]


[[481   0   0]
 [108   0   0]
 [160   0   0]
 ...
 [ 96   0   0]
 [286   5   0]
 [325   0   0]]


In [ ]:
train_df.head()

,ID,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,City,County,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,40.108910,-83.092860,40.112060,-83.031870,3.230,R,481,119,...,1,1,1,1,1,1,1,1,1,1
1,A-2,2,39.865420,-84.062800,39.865010,-84.048730,0.747,R,108,15,...,1,1,1,1,1,1,1,1,1,1
2,A-3,2,39.102660,-84.524680,39.102090,-84.523960,0.055,R,160,43,...,1,1,1,1,1,1,1,1,1,0
3,A-4,2,41.062130,-81.537840,41.062170,-81.535470,0.123,R,561,171,...,1,1,1,1,1,1,1,1,0,0
4,A-5,3,39.172393,-84.492792,39.170476,-84.501798,0.500,R,160,43,...,1,1,1,1,1,1,0,0,0,0


In [ ]:
# Timezone

counties = vectorize_data(train_df["County"])
train_df["County"] = counties[:, 0]

In [ ]:
list_strings = ["State", "Timezone", "Wind_Direction", "Weather_Condition"]

for column in list_strings:
  print("Hello")
  counties = vectorize_data(train_df[column])
  train_df[column] = counties[:, 0]

Hello
Hello
Hello
Hello


In [ ]:
def process_binary(column, class1, class2):
  train_df[column][train_df[column] == class1] = 0
  train_df[column][train_df[column] == class2] = 1

list_binary = ['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop']

for column in list_binary:
  process_binary(column, True, False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
day_night_columns=['Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight']
for column in day_night_columns:
  process_binary(column, "Day", "Night")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
process_binary("Side", "L", "R")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
train_df.to_csv("All_numbers.csv")

# Standardize